In [4]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torchvision import datasets, transforms, models, utils
import torch.nn.functional as F
from torch.autograd import Variable
import time
from glob import glob
from torch.utils.data.sampler import SubsetRandomSampler
from collections import OrderedDict
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [5]:
data_path = "../data/raw/"

In [6]:
annotation_df = pd.read_csv(data_path+'annotation.csv')
annotation_df.head()
print("Annotation data: ",len(annotation_df))

Annotation data:  1026


In [7]:
all_files = glob(data_path+"images/*")
all_jpgs = sorted([img for img in all_files if ".jpg" in img or ".jpeg" in img or "JPG" in img])
print("Total {} photos ".format(len(all_jpgs)))

Total 1026 photos 


In [8]:
id_path = [image for image in all_jpgs ]
image_df = pd.DataFrame(id_path, columns=['path'])

In [10]:
full_df = image_df.merge(annotation_df, left_index=True, right_index=True)
full_df = full_df.drop(['image'], axis=1)
full_df.to_csv("../data/interim/full_annotation.csv", index=False, header=True)
full_df.head()

,path,height,weight,BMI
0,../data/raw/images/f_001.jpg,1.55,61.0,25.390219
1,../data/raw/images/f_002.jpg,1.76,85.0,27.440599
2,../data/raw/images/f_003.jpg,1.78,56.0,17.674536
3,../data/raw/images/f_004.jpg,1.63,63.0,23.711845
4,../data/raw/images/f_005.jpg,1.76,54.0,17.432851


In [11]:
n = 65
image_path = full_df.iloc[n, 0]
values = np.asarray(full_df.iloc[n, 1:])
values = values.astype('float').reshape(-1, 3)

print('Image path: {}'.format(image_path))
print('Values shape: {}'.format(values.shape))
print('height: {} m, weight: {} kg, bmi: {}'.format(*values[:, 0], *values[:, 1], *values[:, 2]))

Image path: ../data/raw/images/f_066.jpg
Values shape: (1, 3)
height: 1.77 m, weight: 55.0 kg, bmi: 17.55561939


In [46]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class FaceToBMIDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=transforms.ToTensor()):
        self.annotaion = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotaion)

    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()
        img_path = self.annotaion.iloc[idx,0]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        values = self.annotaion.iloc[idx, 1:]
        values = np.asarray(values)
        values = values.astype('float').reshape(-1, 3)
        # item = {"image": image, "values": torch.from_numpy(values)}
        # return item
        return image, values

In [47]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

In [48]:
path_to_csv = "../data/interim/full_annotation.csv"
image_dir = "../data/raw/images"
train_data = FaceToBMIDataset(csv_file=path_to_csv, image_dir=image_dir, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

In [49]:
images, values = next(iter(train_loader))
print(type(images))
print(type(values))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [50]:
print(images.shape)
print(values.shape)

torch.Size([16, 3, 224, 224])
torch.Size([16, 1, 3])


In [51]:
print(values)

tensor([[[  1.8200, 102.0000,  30.7934]],

        [[  2.2100, 119.0000,  24.3648]],

        [[  1.8500,  98.0000,  28.6340]],

        [[  1.8000,  73.0000,  22.5309]],

        [[  1.8300,  72.0000,  21.4996]],

        [[  1.6500,  51.0000,  18.7328]],

        [[  1.6800,  58.0000,  20.5499]],

        [[  1.7000,  62.0000,  21.4533]],

        [[  1.8500,  82.0000,  23.9591]],

        [[  1.6300,  58.0000,  21.8300]],

        [[  1.9300,  82.0000,  22.0140]],

        [[  1.7500,  71.0000,  23.1837]],

        [[  1.7800,  78.0000,  24.6181]],

        [[  1.8000,  64.0000,  19.7531]],

        [[  2.1100, 110.0000,  24.7074]],

        [[  2.0300, 113.0000,  27.4212]]], dtype=torch.float64)
